In [2]:
from fcsparser import parse 
import numpy as np
import pandas as pd
import os

In [3]:
root = r"Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16"
raw_cells_location = root + "/FCS_files/cells_panel/"
raw_plasma_location = root + "/FCS_files/plasma_panel/"

# Usage example:
csv_path = root + "/anu_dc_metadata.csv"

In [3]:
import shutil
from typing import List
import logging

def move_fcs_files_to_main_dir(main_dir: str) -> None:
    """Move all .fcs files from subdirectories into the main directory and delete empty subdirectories.

    Args:
        main_dir (str): The path to the main directory containing subdirectories with .fcs files.

    Raises:
        FileNotFoundError: If the main directory does not exist.
        Exception: For any unexpected errors during file operations.

    Example:
        move_fcs_files_to_main_dir(r"Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/FCS_files/cells_panel/")
    """
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    if not os.path.isdir(main_dir):
        raise FileNotFoundError(f"Main directory '{main_dir}' does not exist.")

    # Walk through all subdirectories
    for root_dir, dirs, files in os.walk(main_dir, topdown=False):
        if root_dir == main_dir:
            continue  # Skip the main directory itself

        for file in files:
            if file.lower().endswith('.fcs'):
                src_path = os.path.join(root_dir, file)
                dest_path = os.path.join(main_dir, file)
                # If file with same name exists, rename to avoid overwrite
                if os.path.exists(dest_path):
                    base, ext = os.path.splitext(file)
                    i = 1
                    while os.path.exists(os.path.join(main_dir, f"{base}_{i}{ext}")):
                        i += 1
                    dest_path = os.path.join(main_dir, f"{base}_{i}{ext}")
                    logger.warning(f"File {file} exists in main directory. Renaming to {os.path.basename(dest_path)}.")
                try:
                    shutil.move(src_path, dest_path)
                    logger.info(f"Moved {src_path} to {dest_path}")
                except Exception as e:
                    logger.error(f"Failed to move {src_path} to {dest_path}: {e}")

        # After moving files, remove the subdirectory if empty
        try:
            if not os.listdir(root_dir):
                os.rmdir(root_dir)
                logger.info(f"Removed empty directory: {root_dir}")
        except Exception as e:
            logger.error(f"Failed to remove directory {root_dir}: {e}")

# Usage example:
move_fcs_files_to_main_dir(raw_plasma_location)

In [4]:
from typing import Any
import os
import logging

def remove_spaces_from_filenames(directory: str) -> None:
    """
    Recursively walks through the given directory and its subdirectories,
    renaming all files to remove spaces from their filenames.

    Args:
        directory (str): The root directory to start renaming files.

    Raises:
        FileNotFoundError: If the provided directory does not exist.
        Exception: For any unexpected errors during file operations.

    Example:
        remove_spaces_from_filenames(r"Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/FCS_files/plasma_panel/")
    """
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    if not os.path.isdir(directory):
        raise FileNotFoundError(f"Directory '{directory}' does not exist.")

    for root, _, files in os.walk(directory):
        for filename in files:
            if " " in filename:
                old_path = os.path.join(root, filename)
                new_filename = filename.replace(" ", "")
                new_path = os.path.join(root, new_filename)
                # Avoid overwriting existing files
                if os.path.exists(new_path):
                    base, ext = os.path.splitext(new_filename)
                    i = 1
                    while os.path.exists(os.path.join(root, f"{base}_{i}{ext}")):
                        i += 1
                    new_path = os.path.join(root, f"{base}_{i}{ext}")
                    logger.warning(
                        f"File {new_filename} already exists. Renaming to {os.path.basename(new_path)}."
                    )
                try:
                    os.rename(old_path, new_path)
                    logger.info(f"Renamed '{old_path}' to '{new_path}'")
                except Exception as e:
                    logger.error(f"Failed to rename '{old_path}' to '{new_path}': {e}")

remove_spaces_from_filenames(root)


In [5]:
from typing import Any, Dict, List

def extract_pn_metadata(metadata: Dict[str, Any]) -> List[Any]:
    """
    Extracts all values from the metadata dictionary where the key starts with '$P' and ends with 'N'.

    Args:
        metadata (Dict[str, Any]): The metadata dictionary parsed from an FCS file.

    Returns:
        List[Any]: A list of values corresponding to keys that start with '$P' and end with 'N'.

    Example:
        >>> meta = parse(raw_cells_location + os.listdir(raw_cells_location)[0], meta_data_only=True)
        >>> extract_pn_metadata(meta)
        ['FSC-A', 'SSC-A', ...]
    """
    # List to store the values of keys matching the pattern
    pn_values: List[Any] = [
        value
        for key, value in metadata.items()
        if key.startswith("$P") and key.endswith("N")
    ]
    return pn_values

from typing import Set, Any

def get_shared_pn_names(directory: str) -> Set[Any]:
    """
    Extracts the set of PN names (parameter names) that are shared across all FCS files in the given directory.

    Args:
        directory (str): Path to the directory containing FCS files.

    Returns:
        Set[Any]: Set of PN names shared by all files.

    Raises:
        FileNotFoundError: If the directory does not exist or contains no FCS files.
        Exception: If parsing any file fails.
    
    Example:
        >>> shared_pn_names = get_shared_pn_names(raw_cells_location)
        >>> print(shared_pn_names)
        {'FSC-A', 'SSC-A', ...}
    """
    import os

    # List all FCS files in the directory
    files = [f for f in os.listdir(directory) if f.lower().endswith('.fcs')]

    shared_pn_names: Set[Any] = set()
    for idx, filename in enumerate(files):
        file_path = os.path.join(directory, filename)
        meta = parse(file_path, meta_data_only=True)
        pn_names = set(extract_pn_metadata(meta))
        if idx == 0:
            shared_pn_names = pn_names
        else:
            shared_pn_names &= pn_names

    return shared_pn_names

# Usage example:
cells_shared_pn_names = get_shared_pn_names(raw_cells_location)
plasma_shared_pn_names = get_shared_pn_names(raw_plasma_location)
print(f"PN names shared across all files: {cells_shared_pn_names}")
print(f"PN names shared across all files: {plasma_shared_pn_names}")

PN names shared across all files: {'FSC-W', 'SSC-W', 'Qdot 605-A', 'FITC-A', 'FSC-H', 'Alexa Fluor 700-A', 'Time', 'SSC-H', 'APC-Cy7-A', 'SSC-A', 'PerCP-Cy5-5-A', 'PE-Cy7-A', 'Alexa Fluor 405-A', 'FSC-A', 'Alexa Fluor 430-A', 'PE-A', 'PE-Texas Red-A', 'APC-A'}
PN names shared across all files: {'Alexa Fluor 488-A', 'Qdot 605-A', 'Alexa Fluor 700-A', 'Time', 'APC-Cy7-A', 'SSC-A', 'PE-Cy7-A', '7-AAD-A', 'FSC-A', 'Alexa Fluor 405-A', 'Alexa Fluor 430-A', 'PE-A', 'PE-Texas Red-A', 'APC-A'}


In [16]:
import pandas as pd

markers = pd.DataFrame(columns=list(cells_shared_pn_names))
markers.drop("Time", axis = 1).to_csv(root + "/cells_marker_names.csv", index=False)

markers = pd.DataFrame( columns=list(plasma_shared_pn_names))
markers.drop("Time", axis = 1).to_csv(root + "/plasma_marker_names.csv", index=False)

In [11]:
# adding flowmop markers
import pandas as pd

markers = pd.read_csv(root + "/cells_marker_names.csv")
markers = list(markers.columns )
markers.extend([marker for marker in ["passedlod","passeddebris","passedtime","passeddoublet","passedfinal"]])
pd.DataFrame(columns = markers).to_csv(root + "/cells_marker_names_with_flowmop.csv", index = False)

In [40]:
from typing import List
import os
import pandas as pd

def get_present_files_from_csv(
    csv_path: str, directory: str, filename_column: str = "filename"
) -> List[str]:
    """
    Reads a CSV file and checks which files listed in the specified filename column
    are present in the given directory.

    Args:
        csv_path (str): Path to the CSV file containing file metadata.
        directory (str): Directory to check for file presence.
        filename_column (str): Name of the column in the CSV that contains filenames.

    Returns:
        List[str]: List of filenames that are present in the directory.

    Raises:
        FileNotFoundError: If the CSV file or directory does not exist.
        ValueError: If the filename column is not found in the CSV.

    Example:
        >>> present_files = get_present_files_from_csv(
        ...     "metadata.csv", "Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/FCS_files/cells_panel/"
        ... )
        >>> print(present_files)
        ['file1.fcs', 'file2.fcs']
    """
    # Check if CSV and directory exist
    if not os.path.isfile(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")
    if not os.path.isdir(directory):
        raise FileNotFoundError(f"Directory not found: {directory}")

    # Read CSV
    df = pd.read_csv(csv_path, sep=";")
    if filename_column not in df.columns:
        raise ValueError(f"Column '{filename_column}' not found in CSV.")

    # Normalize directory file list for comparison
    dir_files = set(os.listdir(directory))

    # Check which files from CSV are present in the directory
    present_files = [
        fname for fname in df[filename_column].astype(str) if fname in dir_files
    ]

    return present_files

cells_present_files = get_present_files_from_csv(csv_path, raw_cells_location)
plasma_present_files = get_present_files_from_csv(csv_path, raw_plasma_location)
print(f"Files present in directory: {len(cells_present_files)}")

Files present in directory: 244


In [60]:
import os
from typing import List, Tuple
import pandas as pd
import logging

# Define the columns to keep
COLUMNS_TO_KEEP: List[str] = [
    "filename",
    "tumour_model",
    "day",
    "tumour",
    "treatment",
    "Age (weeks)",
    "Spleen (mg) D8",
    "Spleen (mg) D15",
    "Right_tumour_(mg) D8",
    "Right Tumour (mg) D15",
]

def filter_and_save_metadata(
    csv_path: str,
    present_files: List[str],
    columns_to_keep: List[str],
    suffix: str = "_filtered"
) -> Tuple[str, List[str]]:
    """
    Filter the metadata CSV to only include rows for files present in the directory,
    save the filtered DataFrame with a suffix, and return the filtered CSV path and
    the list of files present in the filtered metadata.

    Args:
        csv_path (str): Path to the original metadata CSV.
        present_files (List[str]): List of filenames present in the directory.
        columns_to_keep (List[str]): List of columns to retain in the filtered CSV.
        suffix (str): Suffix to append to the filtered CSV filename.

    Returns:
        Tuple[str, List[str]]: Path to the filtered CSV file and list of files present in the filtered metadata.

    Raises:
        FileNotFoundError: If the CSV file does not exist.
        ValueError: If required columns are missing.

    Example:
        >>> filtered_csv, filtered_files = filter_and_save_metadata(
        ...     "metadata.csv", ["file1.fcs", "file2.fcs"], ["filename", "treatment"]
        ... )
        >>> print(filtered_csv)
        'metadata_filtered.csv'
        >>> print(filtered_files)
        ['file1.fcs', 'file2.fcs']
    """
    if not os.path.isfile(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")

    # Read the original CSV (assume same separator as before)
    df = pd.read_csv(csv_path, sep=";")

    # Check for required columns
    missing_cols = [col for col in columns_to_keep if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in CSV: {missing_cols}")

    # Filter rows where 'filename' is in present_files
    filtered_df = df[df["filename"].astype(str).isin(present_files)][columns_to_keep]

    # Only keep files that are present in the filtered metadata
    filtered_files: List[str] = filtered_df["filename"].astype(str).tolist()

    # Construct output path
    base, ext = os.path.splitext(csv_path)
    filtered_csv_path = f"{base}{suffix}{ext}"

    # Save filtered DataFrame
    filtered_df.to_csv(filtered_csv_path, sep=";", index=False)
    print(f"Filtered metadata saved to: {filtered_csv_path}")
    print(f"Filtered rows: {len(filtered_df)}")

    return filtered_csv_path, filtered_files

# Usage example:
filtered_cell_csv_path, filtered_cell_files = filter_and_save_metadata(
    csv_path=csv_path,
    present_files=cells_present_files,
    columns_to_keep=COLUMNS_TO_KEEP,
    suffix="_cellsfiltered"
)
# Now, filtered_cell_files contains only the files present in the filtered metadata.
# You can use this list to move/copy files to a separate directory as needed.


Filtered metadata saved to: Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/anu_dc_metadata_cellsfiltered.csv
Filtered rows: 244


In [61]:
import os
import shutil
from typing import List

def copy_files_by_basename(
    source_dir: str,
    basenames: List[str],
    dest_dir: str,
    file_extension: str = ".fcs"
) -> List[str]:
    """
    Copies files from the source directory to the destination directory if their base names match any in the provided list.

    Args:
        source_dir (str): Path to the directory containing the source files.
        basenames (List[str]): List of base filenames (without extension) to match.
        dest_dir (str): Path to the directory where matched files will be copied.
        file_extension (str): File extension to match (default: ".fcs").

    Returns:
        List[str]: List of file paths that were successfully copied.

    Raises:
        FileNotFoundError: If the source directory does not exist.
        OSError: If a file cannot be copied.
    
    Example:
        >>> copy_files_by_basename(
        ...     source_dir="raw_data",
        ...     basenames=["200825_PIIOVo2_FC_4T1R_Bld_D7_C1_003", "20222_MPTLVo4_FC_4T1R_Bld_D14_C5_022"],
        ...     dest_dir="filtered_data"
        ... )
    """
    if not os.path.isdir(source_dir):
        raise FileNotFoundError(f"Source directory not found: {source_dir}")

    os.makedirs(dest_dir, exist_ok=True)
    copied_files: List[str] = []

    # Create a set for faster lookup
    basename_set = set(basenames)

    for fname in os.listdir(source_dir):
        file_base, ext = os.path.splitext(fname)
        if ext.lower() == file_extension.lower() and file_base in basename_set:
            src_path = os.path.join(source_dir, fname)
            dst_path = os.path.join(dest_dir, fname)
            try:
                shutil.copy2(src_path, dst_path)
                copied_files.append(dst_path)
            except OSError as e:
                print(f"Error copying {src_path} to {dst_path}: {e}")

    print(f"Copied {len(copied_files)} files to {dest_dir}")
    return copied_files

# Usage example:
# Suppose you have a list of base filenames (without extension) you want to copy:
# filtered_basenames = [os.path.splitext(f)[0] for f in filtered_cell_files]
# copied = copy_files_by_basename(
#     source_dir="Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/raw_fcs",
#     basenames=filtered_basenames,
#     dest_dir="Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/filtered_fcs"
# )


['200825_PIIOVo2_FC_4T1R_Bld_D7_C1_003.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C1_005.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C1eC100_002.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C1eC1000_004.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C1Nil_001.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C2_009.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C2eC100_006.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C2eC100_010.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C2eC1000_008.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C2Nil_007.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C3_011.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C3_015.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C3eC100_014.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C3eC1000_012.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C3Nil_013.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C4_017.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C4eC100_018.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C4eC1000_016.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C4eC1000_020.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C4Nil_019.fcs',
 '200825_PIIOVo2_FC_4T1R_Bld_D7_C

In [76]:
import os
import re
from typing import List, Optional, Tuple
import logging

import pandas as pd


def parse_filename_for_metadata(
    filename: str,
) -> Optional[Tuple[str, str, str, int]]:
    """
    Parses a filename to extract metadata components for matching.

    The expected filename format is like:
    '200707_MPTLVo2_MacMicLP_4T1R_Bld_D8_C1Nil_001.fcs'

    Args:
        filename (str): The filename to parse.

    Returns:
        Optional[Tuple[str, str, str, int]]: A tuple containing
        (experiment, tumour_model, cage, sample) if parsing is successful,
        otherwise None.
    """
    name_without_ext = os.path.splitext(filename)[0]
    parts = name_without_ext.split("_")

    try:
        # date = parts[0]
        experiment = parts[1]
        tumour_model = parts[3]
        cage_part = parts[6]
        sample = int(parts[7])

        # Extract cage from format like 'C1Nil' -> 'C1'
        cage_match = re.match(r"(C\d+)", cage_part)
        if not cage_match:
            return None
        cage = cage_match.group(1)

        return (experiment, tumour_model, cage, sample)
        # return (date, experiment, tumour_model, cage, sample)
    except (IndexError, ValueError):
        # Handles cases where parts are missing or sample is not an integer
        logging.warning(f"Failed to parse filename: {filename}")
        return None

def filter_and_save_metadata_plasma(
    csv_path: str,
    dir_files: List[str],
    columns_to_keep: List[str],
    suffix: str = "_filtered",
) -> Tuple[str, List[str]]:
    """
    Filter the metadata CSV to include only rows that match files present
    in the directory, based on a complex matching logic, and save the
    filtered DataFrame with a specified suffix.

    Matching is based on experiment, tumour model, cage, and sample
    number parsed from the filename.

    Args:
        csv_path (str): Path to the original metadata CSV.
        dir_files (List[str]): List of filenames present in the directory.
        columns_to_keep (List[str]): List of columns to retain in the filtered CSV.
        suffix (str): Suffix to add to the filtered CSV filename.

    Returns:
        Tuple[str, List[str]]: A tuple containing the path to the filtered
        CSV file and a list of filenames that are present in both the

    Raises:
        FileNotFoundError: If the CSV file does not exist.
        ValueError: If required columns are missing from the CSV.
    """

    df = pd.read_csv(csv_path, sep=";")

    # --- New matching logic ---

    # NOTE: The following column names are assumed to exist in the metadata CSV:
    # 'experiment', 'tumour model', 'cage', 'sample'.
    # If the 'cage' column has a different name, it needs to be updated below.
    matching_cols = ["experiment", "tumour_model", "cage", "sample"]

    # 1. Parse all present filenames and create a mapping from parsed keys to original filenames
    present_files_map = {}
    for file in dir_files:
        key = parse_filename_for_metadata(file)
        if key:
            if key not in present_files_map:
                present_files_map[key] = []
            present_files_map[key].append(file)
    present_file_keys = set(present_files_map.keys())

    # 2. Create a unique identifier tuple for each row in the DataFrame.
    #    Ensure data types are consistent for comparison.
    df_match_subset = df[matching_cols].copy()
    df_match_subset["experiment"] = df_match_subset["experiment"].astype(str)
    df_match_subset["tumour_model"] = df_match_subset["tumour_model"].astype(str)
    df_match_subset["cage"] = df_match_subset["cage"].astype(str)
    df_match_subset["sample"] = df_match_subset["sample"].astype(int)

    df_keys = [tuple(x) for x in df_match_subset.to_numpy()]

    # 3. Create a boolean mask for rows that have a key present in our set
    mask = [key in present_file_keys for key in df_keys]

    # 4. Filter the DataFrame using the mask and select desired columns
    filtered_df = df[mask][columns_to_keep]

    # --- Find matching files ---
    # Get the unique keys that are present in the filtered dataframe
    filtered_keys = {key for i, key in enumerate(df_keys) if mask[i]}

    # Use these keys to find the corresponding filenames
    matching_files = []
    for key in sorted(list(filtered_keys)):
        if key in present_files_map:
            matching_files.extend(present_files_map[key])

    print("matching files len is: ", len(matching_files))

    # --- End of new matching logic ---

    # Construct output path
    base, ext = os.path.splitext(csv_path)
    filtered_csv_path = f"{base}{suffix}{ext}"

    # Save filtered DataFrame
    filtered_df.to_csv(filtered_csv_path, sep=";", index=False)
    print(f"Filtered metadata saved to: {filtered_csv_path}")
    print(f"Filtered rows: {len(filtered_df)}")

    return filtered_csv_path, matching_files

In [77]:
_, filtered_plasma_files = filter_and_save_metadata_plasma(
    csv_path = csv_path,
    dir_files=os.listdir(raw_plasma_location),
    columns_to_keep=COLUMNS_TO_KEEP,
    suffix="_plasmafiltered"
)
 

matching files len is:  341
Filtered metadata saved to: Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16/anu_dc_metadata_plasmafiltered.csv
Filtered rows: 204


In [70]:
import os
import shutil
import logging
from typing import List

def copy_files_by_filename(
    source_dir: str,
    filenames: List[str],
    dest_dir: str,
) -> List[str]:
    """
    Copies files from the source directory to the destination directory if their full filenames match any in the provided list.

    Args:
        source_dir (str): Path to the directory containing the source files.
        filenames (List[str]): List of full filenames (including extension) to match and copy.
        dest_dir (str): Path to the directory where matched files will be copied.

    Returns:
        List[str]: List of file paths that were successfully copied.

    Raises:
        FileNotFoundError: If the source directory does not exist.

    Example:
        >>> copy_files_by_filename(
        ...     source_dir="raw_data",
        ...     filenames=["200825_PIIOVo2_FC_4T1R_Bld_D7_C1_003.fcs", "20222_MPTLVo4_FC_4T1R_Bld_D14_C5_022.fcs"],
        ...     dest_dir="filtered_data"
        ... )
    """
    if not os.path.isdir(source_dir):
        raise FileNotFoundError(f"Source directory not found: {source_dir}")

    os.makedirs(dest_dir, exist_ok=True)
    copied_files: List[str] = []

    # Use a set for O(1) lookup
    filename_set = set(filenames)

    for fname in os.listdir(source_dir):
        if fname in filename_set:
            src_path = os.path.join(source_dir, fname)
            dst_path = os.path.join(dest_dir, fname)
            try:
                shutil.copy2(src_path, dst_path)
                copied_files.append(dst_path)
            except OSError as e:
                logging.warning(f"Error copying {src_path} to {dst_path}: {e}")

    print(f"Copied {len(copied_files)} files to {dest_dir}")
    return copied_files

# Usage example:
# Suppose you have a list of full filenames you want to copy:
# filtered_filenames = filtered_cell_files
copied = copy_files_by_filename(
    source_dir=raw_cells_location,
    filenames=filtered_cell_files,
    dest_dir=os.path.join(root, "FCS_files", "cells_panel_filtered")
)


Copied 244 files to Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16\FCS_files\cells_panel_filtered


In [71]:
copied = copy_files_by_filename(
    source_dir=raw_plasma_location,
    filenames=filtered_plasma_files,
    dest_dir=os.path.join(root, "FCS_files", "plasma_panel_filtered")
)


Copied 341 files to Y:/g/data/eu59/data_flowmop/fig_4_data/ANUDC_16\FCS_files\plasma_panel_filtered
